#### Prompt:
You are an analyst for a premium travel agency. The company has tasked you to analyze the recent bookings they had with some of their clients. Each row represents the transaction of a person of interest (POI) in the company. Some details regarding the POI is also given, such as which luxury group they are from, as well as their individual net worth. Most importantly, you also know their __`route`__ which displays the order of their travels from different points A to E. 

A sample route of __`EBDAC`__ means that:
1. From starting point E, they went to point B.
2. From point B, they went to point D.
3. From point D, they went to point A.
4. From point A, they went to ending point C.

#### You are tasked to come up with the following:
1. How many POIs are there in each group?
2. Using the __`route_matrix.json`__ file, calculate each POI's travel expenditure.
3. Due to company policy, add a column named __`budget`__ to denote if a person is overbudget (if their travel expenses go over 5% of their net worth). If they are overbudget, place them as "Over Budget"; else, place them as "Within Budget".
4. Which group has the __highest average net worth__?
5. Which group has the __highest % of people who are Over Budget__?
6. Which group had the most __direct flights__ from point A to point B (or from B to A)?

In [5]:
# 1. How many POIs are there in each group?
import pandas as pd
import numpy as np

df = pd.read_csv('poi_dataset.csv')

df['group'].value_counts()

group
Group 4     6660
Group 3     6656
Group 9     6645
Group 1     6615
Group 11    6603
Group 12    6600
Group 8     6572
Group 5     6571
Group 18    6565
Group 14    6564
Group 15    6562
Group 13    6558
Group 16    6555
Group 6     6551
Group 19    6546
Group 10    6544
Group 2     6533
Group 20    6517
Group 7     6501
Group 17    6445
Name: count, dtype: int64

In [14]:
# 2. Using the route_matrix.json file, calculate each POI's travel expenditure

jsondf = pd.read_json('route_matrix.json')

def expense(x):
    exp = 0
    for i in range(len(x)-1):
        exp += jsondf.loc[str(x)[i], str(x)[i+1]]
    return exp
        
df['travel_ex'] = df['route'].apply(expense)
df

,poi_number,group,route,net_worth,travel_ex
0,0,Group 1,EBDAC,459,16
1,1,Group 3,ABCED,407,22
2,2,Group 9,BECAD,5,21
3,3,Group 1,BEACD,832,29
4,4,Group 1,CBDAE,928,8
...,...,...,...,...,...
131358,131358,Group 2,AECBD,755,9
131359,131359,Group 9,EADCB,143,11
131360,131360,Group 5,CBAED,202,18
131361,131361,Group 3,ADCEB,277,17


In [40]:
jsondf = pd.read_json('route_matrix.json')

def expense(x):
    exp = 0
    for i in range(len(x)-1):
        exp += jsondf.loc[str(x)[i], str(x)[i+1]]
    return exp
        
df['travel_ex'] = df['route'].apply(lambda x: sum(jsondf.loc[(str(x)[i], str(x)[i+1])] for i in range(len(x)-1)))
df

,poi_number,group,route,net_worth,travel_ex,budget,direct_AB,direct_AB_lambda
0,0,Group 1,EBDAC,459,16,Within Budget,0,0
1,1,Group 3,ABCED,407,22,Over Budget,1,1
2,2,Group 9,BECAD,5,21,Over Budget,0,0
3,3,Group 1,BEACD,832,29,Within Budget,0,0
4,4,Group 1,CBDAE,928,8,Within Budget,0,0
...,...,...,...,...,...,...,...,...
131358,131358,Group 2,AECBD,755,9,Within Budget,0,0
131359,131359,Group 9,EADCB,143,11,Over Budget,0,0
131360,131360,Group 5,CBAED,202,18,Over Budget,1,1
131361,131361,Group 3,ADCEB,277,17,Over Budget,0,0


In [17]:
# 3. Due to company policy, add a column named budget to denote if a person is overbudget (if their travel expenses go over 5% of their net worth). 
# If they are overbudget, place them as "Over Budget"; else, place them as "Within Budget"

df['budget'] = df.apply(lambda x: "Over Budget" if(x['net_worth']*0.05 < x['travel_ex']) else "Within Budget", axis = 1)
df

,poi_number,group,route,net_worth,travel_ex,budget
0,0,Group 1,EBDAC,459,16,Within Budget
1,1,Group 3,ABCED,407,22,Over Budget
2,2,Group 9,BECAD,5,21,Over Budget
3,3,Group 1,BEACD,832,29,Within Budget
4,4,Group 1,CBDAE,928,8,Within Budget
...,...,...,...,...,...,...
131358,131358,Group 2,AECBD,755,9,Within Budget
131359,131359,Group 9,EADCB,143,11,Over Budget
131360,131360,Group 5,CBAED,202,18,Over Budget
131361,131361,Group 3,ADCEB,277,17,Over Budget


In [18]:
# 4. Which group has the highest average net worth?
df.groupby("group").mean(numeric_only= True)["net_worth"].sort_values(ascending = False)


group
Group 10    510.857427
Group 13    506.817627
Group 9     506.086832
Group 4     506.020871
Group 18    505.240061
Group 19    504.931714
Group 20    504.469695
Group 6     503.275836
Group 3     503.217849
Group 17    502.266253
Group 7     501.841870
Group 2     500.622991
Group 5     500.229037
Group 15    499.502133
Group 1     498.621920
Group 16    497.430206
Group 14    496.734918
Group 12    495.620152
Group 8     494.873098
Group 11    493.068605
Name: net_worth, dtype: float64

In [28]:
# 5. Which group has the highest % of people who are Over Budget?

overbudget_df = df[df['budget']== "Over Budget"]
(overbudget_df.groupby(['group', 'budget']).count()['poi_number']/df.groupby(['group']).count()['poi_number']).sort_values(ascending = False)




group     budget     
Group 1   Over Budget    0.367045
Group 12  Over Budget    0.366818
Group 11  Over Budget    0.364683
Group 8   Over Budget    0.364425
Group 15  Over Budget    0.363913
Group 5   Over Budget    0.358697
Group 7   Over Budget    0.358560
Group 14  Over Budget    0.358470
Group 16  Over Budget    0.358352
Group 2   Over Budget    0.357875
Group 13  Over Budget    0.355749
Group 4   Over Budget    0.355405
Group 6   Over Budget    0.355213
Group 9   Over Budget    0.354101
Group 3   Over Budget    0.352314
Group 18  Over Budget    0.351714
Group 17  Over Budget    0.351125
Group 19  Over Budget    0.350749
Group 20  Over Budget    0.350315
Group 10  Over Budget    0.349480
Name: poi_number, dtype: float64

In [34]:
# 6. Which group had the most direct flights from A to B (or from B to A)?

def direct(x):
    if "AB" in x or "BA" in x:
        return 1
    else:
        return 0

df['direct_AB'] = df['route'].apply(direct)
df.groupby('group').sum(numeric_only=True)['direct_AB'].sort_values(ascending =False)

group
Group 3     2742
Group 13    2717
Group 18    2665
Group 12    2659
Group 6     2653
Group 9     2649
Group 19    2640
Group 5     2637
Group 15    2636
Group 2     2629
Group 8     2617
Group 7     2614
Group 1     2611
Group 16    2608
Group 20    2604
Group 10    2602
Group 11    2597
Group 4     2589
Group 14    2574
Group 17    2508
Name: direct_AB, dtype: int64

In [36]:
df['direct_AB_lambda'] = df['route'].apply(lambda x: 1 if("AB" in x or "BA" in x) else 0)
df.groupby('group').sum(numeric_only=True)['direct_AB_lambda'].sort_values(ascending =False)

group
Group 3     2742
Group 13    2717
Group 18    2665
Group 12    2659
Group 6     2653
Group 9     2649
Group 19    2640
Group 5     2637
Group 15    2636
Group 2     2629
Group 8     2617
Group 7     2614
Group 1     2611
Group 16    2608
Group 20    2604
Group 10    2602
Group 11    2597
Group 4     2589
Group 14    2574
Group 17    2508
Name: direct_AB_lambda, dtype: int64

array(['EBDAC', 'ABCED', 'BECAD', 'BEACD', 'CBDAE', 'CADBE', 'ABEDC',
       'ACBDE', 'CEADB', 'CDEAB', 'BDECA', 'EDABC', 'ADBEC', 'BDCEA',
       'AEDBC', 'EDCAB', 'DECAB', 'ACEDB', 'BCAED', 'EDBAC', 'ACEBD',
       'CBAED', 'CABDE', 'BACED', 'EACBD', 'DABCE', 'ECADB', 'EDBCA',
       'ECDAB', 'DABEC', 'CDEBA', 'DAECB', 'DCBAE', 'EBCAD', 'AECBD',
       'DBECA', 'EBACD', 'BEADC', 'DCABE', 'AEDCB', 'BAECD', 'CDBEA',
       'CBDEA', 'ADBCE', 'EDACB', 'EBADC', 'BACDE', 'ADEBC', 'BDAEC',
       'EACDB', 'CDAEB', 'ADECB', 'BDACE', 'DCEAB', 'BEDAC', 'CEABD',
       'BEDCA', 'DEBAC', 'BECDA', 'DCBEA', 'ECBAD', 'BCADE', 'EBCDA',
       'CAEDB', 'CDABE', 'AEBDC', 'CADEB', 'EABDC', 'DBAEC', 'DBEAC',
       'BCDAE', 'CBEAD', 'BCEAD', 'CEDBA', 'ACDEB', 'EBDCA', 'CEBAD',
       'ABDCE', 'CEDAB', 'AECDB', 'DECBA', 'ABECD', 'DACEB', 'EDCBA',
       'DCEBA', 'DEBCA', 'BDCAE', 'DEABC', 'BADEC', 'DBCAE', 'AEBCD',
       'DBACE', 'BDEAC', 'BADCE', 'EADCB', 'DEACB', 'BCDEA', 'BAEDC',
       'DAEBC', 'ADC